In [ ]:
from statistics import mode

import cv2
from tensorflow.keras.models import load_model
import numpy as np

from utils.datasets import get_labels
from utils.inference import detect_faces
from utils.inference import draw_text
from utils.inference import face_box
from utils.inference import apply_offsets
from utils.inference import load_detection_model
from utils.preprocessor import preprocess_input


detection_model_path = 'model_num.xml'
emotion_model_path = 'model_num.hdf5'
emotion_labels = get_labels('fer2013')


frame_window = 10
emotion_offsets = (20, 40)


face_detection = load_detection_model(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile=False)


face_target_size = emotion_classifier.input_shape[1:3]

emotion_window = []

cv2.namedWindow('window_frame')
video_capture = cv2.VideoCapture(0)
while True:
    bgr_image = video_capture.read()[1]
    gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
    faces = detect_faces(face_detection, gray_image)

    for face_coordinates in faces:

        x1, x2, y1, y2 = apply_offsets(face_coordinates, emotion_offsets)
        gface = gray_image[y1:y2, x1:x2]
        try:
            gface = cv2.resize(gface, (face_target_size))
        except:
            continue

        gface = preprocess_input(gface, True)
        gface = np.expand_dims(gface, 0)
        gface = np.expand_dims(gface, -1)
        pred_emotion = emotion_classifier.predict(gface)
        prob_emotion = np.max(pred_emotion)
        emotion_label_arg = np.argmax(pred_emotion)
        emotion_text = emotion_labels[emotion_label_arg]
        emotion_window.append(emotion_text)

        if len(emotion_window) > frame_window:
            emotion_window.pop(0)
        try:
            emotion_mode = mode(emotion_window)
        except:
            continue

        if emotion_text == 'angry':
            color = prob_emotion * np.asarray((255, 0, 0))
        elif emotion_text == 'sad':
            color = prob_emotion * np.asarray((0, 0, 255))
        elif emotion_text == 'happy':
            color = prob_emotion * np.asarray((255, 255, 0))
        elif emotion_text == 'surprise':
            color = prob_emotion * np.asarray((0, 255, 255))
        else:
            color = prob_emotion * np.asarray((0, 255, 0))

        color = color.astype(int)
        color = color.tolist()

        face_box(face_coordinates, rgb_image, color)
        draw_text(face_coordinates, rgb_image, emotion_mode,
                  color, 0, -45, 1, 1)

    bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
    cv2.imshow('window_frame', bgr_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break